# LearningOpenCV-Python

In [2]:
import cv2  # 导入cv2
import numpy as np
import matplotlib.pyplot as plt

## 1.3 Core Operations

### 1.3.1 Basic Operations on Images
1. 获取像素值并修改
2. 获取图像属性
3. 设置Region of Image(ROI)
4. 拆分与合并图像

#### 1. Accessing and Modifying pixel values

In [3]:
img = cv2.imread('../Ch2_Gui_Features/input/Oec.jpeg') # ../ 至上一目录

可以通过像素坐标获取像素值，返回BGR list

In [5]:
px = img[100, 100]
print(px)

[202 182 104]


获取蓝色通道像素值

In [6]:
blue = img[100,100,0] # BGR -> 0, 1, 2
print blue

202


修改像素值

In [7]:
img[100, 100] = [255, 255, 255] # 原处修改
print(img[100, 100])

[255 255 255]


**Better pixel accessing and editing method :**

numpy的 `array.item()` 和 `array.itemset()` 函数对处理单一像素性能更好

In [8]:
img.item(10,10,2) # 获取红色

115

In [9]:
img.itemset((10,10,2),100) # 修改像素值
img.item(10,10,2)

100

#### 2. Accessing Image Properties

img存为Numpy array，需要使用相关Numpy API：[Numpy Manual](http://docs.scipy.org/doc/numpy/)

图像（numpy array）属性包括：像素行列数，通道，图像数据类型，pixel数量......

In [10]:
print(img.shape) # 如果时grayscale图像则只返回像素行列数

(1197L, 1800L, 3L)


In [11]:
print(img.size) # number of pixels

6463800


In [12]:
print(img.dtype) # image datatype

uint8


#### 3. Image ROI

有时候，你将不得不与图像的特定区域打交道。对于人眼检测而言，在图像上首先应执行面部检测，直至图像中的脸部被发现，然后再在该脸部区域内搜索眼睛。这种方法可以提高精度。

In [13]:
fish = img[20:200, 330:500] # numpy slicing
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.imshow('img',fish)
cv2.waitKey(0) 
cv2.destroyAllWindows()

#### 4. Splitting and Merging Image Channels 

BGR通道可以拆分为三个独立的通道（拆出3个颜色通道），而三个通道又可以合并到一起

In [14]:
b, g, r = cv2.split(img)
img = cv2.merge((b,g,r))

或者以：

In [9]:
b, g, r = img[:,:,0], img[:,:,1], img[:,:,2] # 使用更高效的处理

In [15]:
img[:,:,2] = 0 # 设置红色通道array均为0

#### 5. Making Borders for Images (Padding)

使用`cv2.copyMakeBoder()`函数（此外该函数可以做convolution operation, zero padding）

主要参数：
- **src** - 输入图形img
- **top, bottom, left, right** - 对应方向的边界宽度，以pixel定义
- **borderType** - 定义添加的边界类型，可以是以下各类:
  - **cv2.BORDER_CONSTANT** - Adds a constant colored border. The value should be given as next argument.
  - **cv2.BORDER_REFLECT** - Border will be mirror reflection of the border elements, like this : fedcba|abcdefgh|hgfedcb
  - **cv2.BORDER_REFLECT_101** or **cv2.BORDER_DEFAULT** - Same as above, but with a slight change, like this : gfedcb|abcdefgh|gfedcba
  - **cv2.BORDER_REPLICATE** - Last element is replicated throughout, like this: aaaaaa|abcdefgh|hhhhhhh
  - **cv2.BORDER_WRAP** - Can’t explain, it will look like this : cdefgh|abcdefgh|abcdefg
- value - 边界颜色（若边界类型为cv2.BORDER_CONSTANT）

In [16]:
BLUE = [255,0,0]
img1 = cv2.imread('./input/opencv_logo.png')
replicate = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_REPLICATE)
reflect = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_REFLECT)
reflect101 = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_REFLECT_101)
wrap = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_WRAP)
constant= cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_CONSTANT,value=BLUE)
plt.subplot(231),plt.imshow(img1,'gray'),plt.title('ORIGINAL')
plt.subplot(232),plt.imshow(replicate,'gray'),plt.title('REPLICATE')
plt.subplot(233),plt.imshow(reflect,'gray'),plt.title('REFLECT')
plt.subplot(234),plt.imshow(reflect101,'gray'),plt.title('REFLECT_101')
plt.subplot(235),plt.imshow(wrap,'gray'),plt.title('WRAP')
plt.subplot(236),plt.imshow(constant,'gray'),plt.title('CONSTANT')
plt.show() # Image is displayed with matplotlib. So RED and BLUE planes will be interchanged

### 1.3.2 Arithmetic Operations on Images Goal
1. 学习若干简单图像计算算法，如：加，减，按位计算等等
2. 函数cv2.add(); cv2.addWeighted()

#### 1. Image Addition
采用 `cv2.add()` 或者 `res = img1 + img2` 方式，但要注意：
>OpenCV addition is a saturated operation while Numpy addition is a modulo operation.

例如：

In [17]:
x = np.uint8([250])
y = np.uint8([10])
print(cv2.add(x, y)) # 250+10 = 260 => 255
print(x + y) # 250+10 = 260 - 256 = 4

[[255]]
[4]


#### 2. Image Blending
>图像合成计算公式：$g(x)=(1-\alpha)f_0(x)+ \alpha f_1(x) + \gamma$

In [4]:
img1 = cv2.imread('./input/ml.jpg')
img2 = cv2.imread('./input/opencv_logo.png') # 两张图的像素行列要一样(90x90像素)
dst = cv2.addWeighted(img1, 0.7, img2, 0.3, 0)
# cv2.namedWindow('dst',cv2.WINDOW_NORMAL)
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 3. Bitwise Operations
包括 `AND`; `OR`; `NOT` 以及` XOR`操作

在提取图像的任何部分区域时（我们将在接下来的章节中看到的）他们将是非常有用的，例如定义和使用非矩形ROI。

**以下例子将展示如何改变图像的特定区域：**

我希望实现将OpenCV的标志放置在某张图像的上方。如果我通过add方法处理两个图像，这会改变显示的色彩。但如果我将其融入到image中，则会有一个透明的效果。但我不希望它是透明的。如果这是一个矩形区域，我可以用前述的ROI来处理。但是OpenCV的标志不是一个矩形，而是非规则的。所以，你可以用如下的按位运算来做到这一点：

In [5]:
# Load two images
img1 = cv2.imread('./input/messi5.jpg')
img2 = cv2.imread('./input/opencv_logo.png')
# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]
# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY) # 将image转为灰度图
cv2.imshow('img2gray',img2gray)
# ----------------------
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY) # 阈值选区
cv2.imshow('mask',mask)
# ----------------------
mask_inv = cv2.bitwise_not(mask) # 反选处logo(即logo区变白)
# ----------------------
cv2.imshow('mask_inv',mask_inv)
# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst
cv2.imshow('res',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 1.3.3 Performance Measurement and Improvement Techniques
**在图像处理，因为你需要每秒处理大量的运算。强制性要求：你的代码不仅能提供正确的解决方案，而且在以最快的速度运行。**
1. 评估代码性能
2. 对于提高性能的一些建议
3. 主要函数 `cv2.getTickCount`；`cv2.getTickFrequency`

此外Python也提供了 **time** 模块，对于评估代码运行速度也是有帮助的，**profile**模块则提供了代码的详细报告，如每个函数所消耗的时间，函数被调用的次数等等。但是如果你使用Ipython，那么这些功能都集成在用户友好操作中了。

#### 1. Measuring Performance with OpenCV
- `cv2.getTickCount()`  函数返回时钟周期的数目，reference events（如机器被接通）的发生到此函数被调用时刻的计数值。
- `cv2.getTickFrequency` 函数返回时钟频率，或者是每秒多少个时钟周期，所以要想了解执行时间你可以这么做：

In [4]:
e1 = cv2.getTickCount()
# your code execution
e2 = cv2.getTickCount()
time = (e2 - e1)/cv2.getTickFrequency()
print(time)

3.8683665901e-05


In [6]:
img1 = cv2.imread('./input/messi5.jpg')
e1 = cv2.getTickCount()
for i in xrange(5,49,2):
    img1 = cv2.medianBlur(img1,i) # 运用中值滤波
e2 = cv2.getTickCount()
t = (e2 - e1)/cv2.getTickFrequency()
print(t)
cv2.imshow('res',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.62684235893


#### 2. Default Optimization in OpenCV
许多OpenCV的函数是使用SSE2，AVX等优化过的。其中也包含了一些未优化过的代码。OpenCV会自动运行优化过的代码如果使能的话。
- `cv2.useOptimized()` ：检测是否打开优化
- `cv2.setUseOptimized()` ：使能

In [9]:
# check if optimization is enabled
cv2.useOptimized() # enabled by default

True

In [11]:
img = cv2.imread('./input/messi5.jpg')
%timeit res = cv2.medianBlur(img,49)

10 loops, best of 3: 25.7 ms per loop


Disable Optimization：

In [10]:
'''cv2.setUseOptimized(False) # Disable it'''

'cv2.setUseOptimized(False) # Disable it'

In [13]:
img = cv2.imread('./input/messi5.jpg')
# optimized median filtering is almost ~2x faster than unoptimized version
%timeit res = cv2.medianBlur(img,49) 

100000 loops, best of 3: 16.4 µs per loop


#### 3. Measuring Performance in IPython
Ipython提供 `%timeit` ，多次运行代码以评估平均运行时间。同样的，其适用于单行代码

In [14]:
x = 5
%timeit y=x**2

10000000 loops, best of 3: 54.3 ns per loop


In [15]:
%timeit y=x*x

10000000 loops, best of 3: 51.2 ns per loop


In [4]:
z = np.uint8([5])
%timeit y=z*z # Python scalar operations are faster than Numpy scalar operations

The slowest run took 39.90 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 544 ns per loop


In [5]:
%timeit y=np.square(z) 

The slowest run took 24.48 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 613 ns per loop


compare the performance of `cv2.countNonZero()` and `np.count_nonzero()` for same image

In [9]:
img = cv2.imread('./input/messi5.jpg', 0) # grayscale
%timeit z = cv2.countNonZero(img)

100000 loops, best of 3: 12.9 µs per loop


In [8]:
img = cv2.imread('./input/messi5.jpg', 0) # grayscale
%timeit z = np.count_nonzero(img)

1000 loops, best of 3: 383 µs per loop


通常情况下，OpenCV函数比numpy的函数更快。因此，对于相同的操作，**OpenCV函数是优选的**。但是，也有例外，尤其是Numpy以**view**方式工作而非copies（复制原数据）。

**以下为注意点：**
1. **Avoid using loops** in Python as far as possible, especially double/triple loops etc. They are inherently slow.
2. Vectorize the algorithm/code to the maximum possible extent because **Numpy and OpenCV are optimized for
vector operations**.
3. Exploit the cache coherence.
4. **Never make copies of array** unless it is needed. Try to use views instead. Array copying is a costly operation.

Even after doing all these operations, if your code is still slow, or use of large loops are inevitable, use additional
libraries like **Cython** to make it faster.

#### 1.3.4 Mathematical Tools in OpenCV
主成分分析(**PCA**)与奇异值分解(**SVD**)